In [1]:
import numpy as np
import gzip
import pandas as pd
import pickle
import os
import random
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler
from torch.autograd import Variable

In [2]:
def normalizeData(x, length=128):
    print('Normalizing: ', x.shape)
    for i in range(x.shape[0]):
        x[i,:,0] = x[i,:,0]/np.linalg.norm(x[i,:,0], 2)
    return x

#function to change amplitude to phase
def amplitudeToPhase(x, length=128):
    xComplex = x[:,0,:] + 1j*x[:,1,:]
    xAmplitude = np.abs(xComplex)
    xAngle = np.arctan2(x[:,1,:],x[:,0,:])/np.pi
    xAmplitude = np.reshape(xAmplitude, (-1,1,length))
    xAngle = np.reshape(xAngle, (-1,1,length))
    x = np.concatenate((xAmplitude, xAngle), axis = 1)
    x = np.transpose(np.array(x), (0,2,1))
    return x

#function to create datalaoder
def dataloader(fileLocation):
    with open(fileLocation, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        p = u.load()
    
    snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], p.keys())))), [1,0])
    X = []
    label = []
    SNRs = []
    for mod in mods:
        for snr in snrs:
            X.append(p[(mod, snr)])
            for i in range(p[(mod,snr)].shape[0]):
                label.append(mod)
                SNRs.append(snr)
    X = np.vstack(X)
    encoder = LabelEncoder()
    encoder.fit(label)
    label = encoder.transform(label)

    X = amplitudeToPhase(X, length=128)
    X = normalizeData(X, length=128)

    #creating training, testing and validation set
    #training set contains 70% of the total data, validation and test set contains 15% each
    #SNRs are also split so as to check the classification accuracy of each SNR
    xTrain, xTest, yTrain, yTest, snrTrain, snrTest = train_test_split(X, label, SNRs, test_size=0.99995, shuffle=True)
   
    return xTrain, xTest, yTrain, yTest, snrTrain, snrTest

In [3]:
xTrain, xTest, yTrain, yTest, snrTrain, snrTest = dataloader('D:\\Downloads\\RadioML\\radioML.pkl')

Normalizing:  (220000, 128, 2)


In [21]:
#creates training triplet dataset
class TripletRadioMLTrain(Dataset):
    def __init__(self, xTrain, yTrain):
        self.xTrain = xTrain
        self.yTrain = yTrain
        self.labels_set = set(self.yTrain)
        self.label_to_indices = {label: np.where(self.yTrain == label)[0] for label in self.labels_set}
    def __getitem__(self, index):
        signal1, label1 = self.xTrain[index], self.yTrain[index].item()
        positive_index = index
        while positive_index == index:
            positive_index = np.random.choice(self.label_to_indices[label1])
        negative_label = np.random.choice(list(self.labels_set - set([label1])))
        negative_index = np.random.choice(self.label_to_indices[negative_label])
        signal2 = self.xTrain[positive_index]
        signal3 = self.xTrain[negative_index]
        return (signal1, signal2, signal3), []
    def __len__(self):
        return len(self.xTrain)        

In [22]:
#creates testing triplet dataset
class TripletRadioMLTest(Dataset):
    def __init__(self, xTest, yTest):
        self.xTest = xTest
        self.yTest = yTest
        self.labels_set = set(self.yTest)
        self.label_to_indices = {label: np.where(self.yTest == label)[0] for label in self.labels_set}
        random_state = np.random.RandomState(29)
        triplets = [[i, random_state.choice(self.label_to_indices[self.yTest[i].item()]), random_state.choice(self.label_to_indices[np.random.choice(list(self.labels_set - set([self.yTest[i].item()])))])] for i in range(len(self.yTest))]
        self.test_triplets = triplets
    def __getitem__(self, index):
        signal1 = self.xTest[self.test_triplets[index][0]]
        singal2 = self.xTest[self.test_triplets[index][1]]
        signal3 = self.xTest[self.test_triplets[index][2]]
        return (signal1, signal2, signal3), []
    def __len__(self):
        return len(self.xTest)

In [23]:
tripletTrainDataset = TripletRadioMLTrain(xTrain, yTrain)
tripletTestDataset = TripletRadioMLTest(xTest, yTest)

In [24]:
batch_size = 128
cuda = torch.cuda.is_available()
triplet_train_loader = torch.utils.data.DataLoader(tripletTrainDataset, batch_size=batch_size, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(tripletTestDataset, batch_size=batch_size, shuffle=True)

In [25]:
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(nn.LSTM(128, 128, 2, batch_first=True),
                                    nn.ReLU(),
                                    torch.nn.Dropout(p = 0.2))

        self.fc = nn.Sequential(nn.Linear(128, 64),
                                nn.PReLU(),
                                nn.Linear(64, 11)
                                )

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

    def get_embedding(self, x):
        return self.forward(x)

In [26]:
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2, x3):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        output3 = self.embedding_net(x3)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

In [27]:
class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean() if size_average else losses.sum()

In [28]:
def fit(train_loader, val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[],
        start_epoch=0):
    """
    Loaders, model, loss function and metrics should work together for a given task,
    i.e. The model should be able to process data output of loaders,
    loss function should process target output of loaders and outputs from the model
    Examples: Classification: batch loader, classification model, NLL loss, accuracy metric
    Siamese network: Siamese loader, siamese model, contrastive loss
    Online triplet learning: batch loader, embedding model, online triplet loss
    """
    for epoch in range(0, start_epoch):
        scheduler.step()

    for epoch in range(start_epoch, n_epochs):
        scheduler.step()

        # Train stage
        train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)

        message = 'Epoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch + 1, n_epochs, train_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        val_loss, metrics = test_epoch(val_loader, model, loss_fn, cuda, metrics)
        val_loss /= len(val_loader)

        message += '\nEpoch: {}/{}. Validation set: Average loss: {:.4f}'.format(epoch + 1, n_epochs,
                                                                                 val_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        print(message)


def train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics):
    for metric in metrics:
        metric.reset()

    model.train()
    losses = []
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        target = target if len(target) > 0 else None
        if not type(data) in (tuple, list):
            data = (data,)
        if cuda:
            data = tuple(d.cuda() for d in data)
            if target is not None:
                target = target.cuda()


        optimizer.zero_grad()
        outputs = model(*data)

        if type(outputs) not in (tuple, list):
            outputs = (outputs,)

        loss_inputs = outputs
        if target is not None:
            target = (target,)
            loss_inputs += target

        loss_outputs = loss_fn(*loss_inputs)
        loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
        losses.append(loss.item())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        for metric in metrics:
            metric(outputs, target, loss_outputs)

        if batch_idx % log_interval == 0:
            message = 'Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data[0]), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), np.mean(losses))
            for metric in metrics:
                message += '\t{}: {}'.format(metric.name(), metric.value())

            print(message)
            losses = []

    total_loss /= (batch_idx + 1)
    return total_loss, metrics


def test_epoch(val_loader, model, loss_fn, cuda, metrics):
    with torch.no_grad():
        for metric in metrics:
            metric.reset()
        model.eval()
        val_loss = 0
        for batch_idx, (data, target) in enumerate(val_loader):
            target = target if len(target) > 0 else None
            if not type(data) in (tuple, list):
                data = (data,)
            if cuda:
                data = tuple(d.cuda() for d in data)
                if target is not None:
                    target = target.cuda()

            outputs = model(*data)

            if type(outputs) not in (tuple, list):
                outputs = (outputs,)
            loss_inputs = outputs
            if target is not None:
                target = (target,)
                loss_inputs += target

            loss_outputs = loss_fn(*loss_inputs)
            loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
            val_loss += loss.item()

            for metric in metrics:
                metric(outputs, target, loss_outputs)

    return val_loss, metrics

In [29]:
margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

In [ ]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)